In [1]:
import ipywidgets as widgets
from IPython.display import display

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Importo funciones del directorio src
import src.toolbox as tb
import src.limpieza as l

# Cargo el dataframe
df = pd.read_csv("input/covid_19_containment_measures_data.csv")

# Aplico los pasos de limpieza que he seguido en el notebook 01-limpieza usando algunas funciones
df = l.drop_nan_by_field(df, 'Country')
df = l.divide_csv_field(df, 'Keywords')
df = l.df_select_columns(df, "Country", "Date Start", "Date end intended", "Keywords")
df['Date Start'] = pd.to_datetime(df['Date Start'], format='%b %d, %Y')
df['Country'] = df['Country'].apply(lambda c: "United States" if c.startswith("US:") else c)
df['Keywords'] = df['Keywords'].apply(lambda s: s.strip())
df['Keywords'] = df['Keywords'].apply(l.normalize_keyword_field)

print("Setup completado!")

Setup completado!


# Dropdown de Paises:
Medidas adoptadas por cada país.

In [2]:
ALL = 'ALL'

# Creo un dropdown y le paso los valores de la columna Country.
dropdown_country = widgets.Dropdown(options = tb.unique_sorted_values_plus_ALL(df['Country']))

# Creo un objeto Output para gestionar la salida de datos.
output_country = widgets.Output()

# Creo un event handler para gestionar el cambio de item en el dropdown.
def dropdown_country_eventhandler(change):
    output_country.clear_output()
    with output_country:
        if (change.new == ALL):
            # Muestro solo los cinco primeros valores (para no saturar el notebook)
            display(df.head())
        else:
            # Muestro los valores unicos de Keywords
            display(df[df["Country"] == change.new].drop_duplicates(subset=['Keywords']).sort_values(by="Date Start"))

# Asocia el event handler al dropdown
dropdown_country.observe(dropdown_country_eventhandler, names='value')

In [3]:
# Muestra el dropdown
display(dropdown_country)

# Muestra el output (sin esto y el código que hemos puesto al principio del event handler los resultados se apiñan uno detras de otro)
display(output_country)

Dropdown(options=('ALL', 'Afghanistan', 'Albania', 'Andorra', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',…

Output()

# Selección por rango de fechas

In [55]:
# Defino las fechas limite partiendo de la primera y ultima del dataframe
start_date = min(df['Date Start'])
end_date = max(df['Date Start'])

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %Y-%m-%d '), date) for date in dates]
index = (0, len(options)-1)

# Con las opciones calculadas arriba creo un SelectionRangeSlider
date_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Fechas',
    orientation='horizontal',
    layout={'width': '500px'}
)

# Creo un objeto Output para gestionar la salida de datos del grafico.
plot_output = widgets.Output()

# Creo un event handler para gestionar el cambio en los rangos de fechas.
def date_slider_eventhandler(change):
    plot_output.clear_output()
    
    with plot_output:
        fecha_inferior, fecha_superior = change.new
        #df[(df['Keywords'] != "first case") & (df["Date Start"] >= fecha_inferior) & (df["Date Start"] <= fecha_superior)]['Country'].head().value_counts().plot.bar()
        #df[(df['Keywords'] != "first case") & (df["Date Start"] >= fecha_inferior) & (df["Date Start"] <= fecha_superior)]['Country'].head().value_counts()
        display(df[(df["Date Start"] >= fecha_inferior) & (df["Date Start"] <= fecha_superior)])
        
# Asocio el event handler al slider de fechas
date_slider.observe(date_slider_eventhandler, names='value')

In [56]:
# Muestro el slider
display(date_slider)

# Muestra el output
display(plot_output)

SelectionRangeSlider(description='Fechas', index=(0, 119), layout=Layout(width='500px'), options=((' 2019-12-1…

Output()